In [2]:
def spazio():
    print("\n")
    print("------------------------------------")
    print("\n")

Costruiamo un loop di addestramento: partiamo dalla struttura del loop, perchè dopo andiamo a definire tutte le entità che vanno a definire l'addestramento di una nn con pytorch.
STRUTTURA GENERALE

In [3]:
import numpy as np
import torch 
from torchvision import datasets #modellazione dataset più famosi
from torchvision import transforms 
from torch.utils.data import DataLoader
from torch import nn
#Oggetto che modella il dataset: servirà per accedere ai dati su disco: Project specific.

training_dataset= datasets.FashionMNIST(
    root="data", #dove salvare i dati, si collega ad un URL remoto
    train=True, #se è un dataset di training o di test
    download=True, #se voglio scaricare i dati
    transform=transforms.ToTensor(), 
)

test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)


'''
Mentre il dataset servirà ad avere un riferimento a dove per esempio si trovano le immagini (path e directory)
Vi offre delle funzioni per caricare delle immagini da disco ecc..

Il dataloader si preoccupa di generare le batches, e quindi di campionare le batch dal dataset e poi di 
mischiare il dataset al termine di ogni epoca.

'''
#Strumento 
train_dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True) 
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


#modello: la rete e la sua architettura, la dobbiamo creare noi.
#Best practice è istanziare una classe che si estende da una classe base di pytorch.

class NeuralNetwork(nn.Module): 
    def __init__(self):
        super(NeuralNetwork, self).__init__() #Costruttore della classe module che è quella base
        
        #dobbiamo istanziare i layers:
        self.flatten = nn.Flatten() #Flatten è un layer che trasforma il tensore in un vettore. Prende img e crea vettore. 
        self.fc1 = nn.Linear(28*28, 512) #Dimensionalità dell'input è 28*28, 512 sono i neuroni nel primo hidden layer. NB: sono vettori.
        self.relu = nn.ReLU() 
        self.fc2 = nn.Linear(512, 10) #Dimensione di iput è 512, 10 sono le classi.

    #Adesso implemento la forward che è la funzione che spiega come la rete processa il dato fino all'output. Il Framework fa la backward.
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = NeuralNetwork()


#Hyperparameters che serviranno per l'addestramento.
learning_rate = 1e-3
batch_size = 64
epochs = 5


#Loss function: che deve essere defintia. in realtà da scegliere e impende dal task.

loss_fn = torch.nn.CrossEntropyLoss() #per problemi di classificazione

#Optimizer: che deve essere definito. in realtà scelto
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #per come costruiremo model avrà una funzione che si chiama paarameters() che restituisce la lista di tutti i parametri.

A questo punto abbiamo tutto l'occorrente per creare il nostro LOOP di addestramento che a grandi linee avrà sempre lo stesso schema:
     - Iterazione per ogni Epoca.
     - Per ogi Epoca:devo iterare per ogni batch.
     - Per ogni batch:chiedo di fare il forward e calcolo la loss.Inoltre faccio la backpropagation, inizialmente azzerando i gradienti perchè di defaul l'ottimizzatore accumula i gradienti di più iterazioni. Poi calcolo la loss.backward() che dovrebbe servire per aggiornare i pesi. Infine ottimizzatore.step().
     -terminata un'epoca, il dataloader viene resettato, e viengono rimischiati tutti i dati. 
     -ricomincia una nuova epoca, riparte tutta la storiella. 



In [ ]:
#BLOCCO STANDARD

for epoch in range(epochs):
    for batch_indx, (X, y ) in enumerate(train_dataloader): #train_dataloader fornisce la tupla samples e labels. Enumerates mi riporta l'indice dell'iterazione.
         
         prediction = model.forward(X) #forward pass
         loss = loss_fn(prediction, y) #calcolo loss

         #backpropagation
         optimizer.zero_grad()
         loss.backward() #Calcolo i gradienti 
         optimizer.step() #Applica la formula per esempio di SGD. In pratica qui avviene la modifica dei parametri aggiornandoli con quei gradienti. w - alphaa*grad


         if batch_indx % 10 == 0:
            print(f"loss: {loss.item()}")

    
    '''
    Nessuno ci vieta alla fine di ogni epoca di fare un test sul test set oppure su validation set
    Lo si fa utilizzando la funzione with torch.no_grad(, poichè qua non abbiamo bisogno di calcolare i gradienti.
    La sintassi dice che tutto quello che è nel blocco which ha no_grad settato.

    '''
    correct = 0
    with torch.no_grad():
        for batch_indx, (X, y) in enumerate(test_dataloader):
            prediction = model(X)  #forward pass anche se non l'ho scritto model.forward(X)è un alias.
            loss = loss_fn(prediction, y)
            correct += (prediction.argmax(1) == y).type(torch.float).sum().item() #below
            print(f"loss: {loss.item()}")

    #Una volta fatto per tutto il dataset di accuracy possiamo printare la nostra accuracy:
    print(f"Accuracy: {100*(correct/len(test_dataset))}%")


correct += (prediction.argmax(1) == y).type(torch.float).sum().item()

Siccome il problema è di classificazione, abbiamo delle probabilità per ogni classe. Ovvero le probabilità che quel campione appartenga a quella classe.
Dobbiamo selezionare l'elemento tra le varie classi che ha la probabilità più alta.
Se questo è uguale a y, cioè l'etichetta di supervisione associata, allora effettuo la somma e quindi ho classificato correttamente.
ho forzato a tipo float e .item() mi fornisce il valore numerico.

ATTENZIONE: NELLO SCRIPT QUI SOPRA ABBIAMO CHE IL DATASET VIENE DIRETTAMENTE SCARICATO TRAMITE UN COMANDO DI PYTORCH. QUESTO PERCHE' IN PYTORCH SONO SALVATI I DATASETS PIU' FAMOSI. 

IN GENERALE QUESTO NON MI CAPITERà MAI! QUELLO CHE DOVRO' FARE SARA' COSTRUIRE UN OGGETTO DATASET CUSTOM.
''' -E' impoertante che oggetto Datasset abbia una sorta dilink con pytorch, cioè che la classe che andiamo ad implementare che rappresenta il nostro dataset
    deve ereditare una classe case di pytorch, che rappresente un dataset genrico, una classe astratta generica. Dobbiamo estendere una classe di base di torch.

    -Ereditare da una classe mi da dei vincoli su cosa posso fare su questa nuova classe. 
    In particolare quando creiamo un nuovo oggetto che rappresenta il nostro Dataset, dobbiamo implementare almeno queste tre funzioni: init (costruttore)

'''



In [ ]:
import pandas as pd
from torch.utils.data import Dataset #adesso posso creare la mia classe 
import os
from torchvision.io import read_image

'''
Idealmente sto immaginando che il mio dataset sia una cartella di immagini, presente nel mio hard disk (sarà un path), e che associata a questa cartella 
vi sia un file di annotazioni (annotation file) nel quale sono presenti le etichette. 

img.001.jpg ---> etichetta 
ecc..

E' possibile anche avere immagini in tre cartelle differenti, pertanto al costruttore dovrò passare 3 paths. Questa è la parte più custom

Supponiamo per esempio che il nostro file annotation sia della forma csv:
00001.jpg, 0
00002.jpg, 1
00003.jpg, 0
00004.jpg, 2
...

'''

class MyDataset(Dataset):                                             #Dataset è la classe di base da cui si eredità 
    def __init__(self , annotation_file , img_dir, transform = None):
        self.annotation_lables = pd.read_csv(annotation_file)         # il file di annotazioni, ed è una lista lunga tanto quanti sono i numeri dei campioni.
        self.img_dir = img_dir                                        # directory delle immagini
        self.transform = transform                                    # trasformazioni da applicare

    def __len__(self):                                                #ritorna la lunghezza del dataset, servirà al dataloader per iterare sul dataset.
        return len(self.img_labels)                                   #ATTENZIONE: come si calcola questa lunghezza dipende dal caso specifico.
    
    def __getitem__(self, index): #E' la funzione che il dataloader chiama  quando deve costruire le batches. se batch size=64 allora questa funzione è chiamata 64 volte.
        
        #Devo tirare fuori una tupla (img.jpg, label)
        img_path = os.join(self.img_dir, self.img_labels.iloc[index, 0]) #punto all'immagine in img_dir e gli assegno il nome che sta sul file annotation_lables

        #Devo leggere immagine da disco
        image = read_image(img_path)

        #devo definire la lable:
        label = self.img_labels.iloc[index, 1]

        if self.transform:
            image = self.transform(image)
        return image, label
    

